In [17]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPool2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
import kt_utils 

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = kt_utils.load_dataset()

X_train = X_train_orig / 255
X_test = X_test_orig / 255

Y_train = Y_train_orig.T
Y_test = Y_test_orig.T


print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

def Happy_model(input_shape):
    #定义一个placeholder，维度input_shape
    X_input = Input(input_shape)
    
    X = ZeroPadding2D((3, 3))(X_input)
    
    #对X使用 CONV -> BN -> RELU 块
    X = Conv2D(32, (7, 7), strides = (1, 1), name='conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)
    
    #最大池化层
    X = MaxPooling2D((2, 2), name = 'max_pool')(X)
    
    #降维
    X = Flatten()(X)
    #全连接层FC
    X = Dense(1, activation='sigmoid')(X)
    
    model = Model(inputs = X_input, outputs = X, name = 'HappyFaceModel')
    
    return model

happy_model = Happy_model(X_train.shape[1:])
happy_model.compile("adam", "binary_crossentropy", metrics=['accuracy'])
happy_model.fit(X_train, Y_train, epochs=40, batch_size=50)

preds = happy_model.evaluate(X_test, Y_test, batch_size=32, verbose=1, sample_weight=None)
print ("误差值 = " + str(preds[0]))
print ("准确度 = " + str(preds[1]))


number of training examples = 600
number of test examples = 150
X_train shape: (600, 64, 64, 3)
Y_train shape: (600, 1)
X_test shape: (150, 64, 64, 3)
Y_test shape: (150, 1)
Epoch 1/40
12/12 [==============================] - 87s 66ms/step - loss: 2.4965 - accuracy: 0.5367
Epoch 2/40
12/12 [==============================] - 0s 24ms/step - loss: 0.4878 - accuracy: 0.7900
Epoch 3/40
12/12 [==============================] - 0s 24ms/step - loss: 0.2635 - accuracy: 0.8850
Epoch 4/40
12/12 [==============================] - 0s 24ms/step - loss: 0.1638 - accuracy: 0.9367
Epoch 5/40
12/12 [==============================] - 0s 24ms/step - loss: 0.1480 - accuracy: 0.9467
Epoch 6/40
12/12 [==============================] - 0s 24ms/step - loss: 0.1122 - accuracy: 0.9650
Epoch 7/40
12/12 [==============================] - 0s 25ms/step - loss: 0.0970 - accuracy: 0.9650
Epoch 8/40
12/12 [==============================] - 0s 24ms/step - loss: 0.0872 - accuracy: 0.9783
Epoch 9/40
12/12 [===============

out = [0.        0.        1.3454674 2.0318177 0.        1.3246754]


In [38]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    功能：
    实现跳跃三层的恒等快
    参数：
    X:输入数据,维度(m, n_H_prev, n_W_prev, n_C_prev)
    f:第二部分卷积层的维度
    filters:每部分卷积层的过滤器个数
    stage:命名参数，层数
    block:命名参数，层的名字
    s:第一部分、第三部分、X_shortcut的stride
    返回值：
    X_out:恒等快输出，维度(n_H, n_W, n_C)
    """
    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base = "bn" + str(stage) + block + "_branch"
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    #第一部分
    X = Conv2D(filters = F1, 
               kernel_size=(1,1),
               strides=(s,s),
               padding="valid",
               name=conv_name_base+"2a",
               kernel_initializer=glorot_uniform(seed=0)
              )(X)
    X = BatchNormalization(axis=3, name=bn_name_base+"2a")(X)
    X = Activation("relu")(X)
    
    #第二部分
    X = Conv2D(filters = F2, 
               kernel_size=(f,f),
               strides=(1,1),
               padding="same",
               name=conv_name_base+"2b",
               kernel_initializer=glorot_uniform(seed=0)
              )(X)
    X = BatchNormalization(axis=3, name=bn_name_base+"2b")(X)
    X = Activation("relu")(X)
    
    #第三部分
    X = Conv2D(filters = F3, 
               kernel_size=(1,1),
               strides=(1,1),
               padding="valid",
               name=conv_name_base+"2c",
               kernel_initializer=glorot_uniform(seed=0)
              )(X)
    X = BatchNormalization(axis=3, name=bn_name_base+"2c")(X)
    
    #shortcut
    X_shortcut = Conv2D(filters = F3, 
               kernel_size=(1,1),
               strides=(s,s),
               padding="valid",
               name=conv_name_base+"1",
               kernel_initializer=glorot_uniform(seed=0)
              )(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base+"1")(X_shortcut)
    
    #求和
    X = Add()([X, X_shortcut])
    #激活
    X_output = Activation("relu")(X)
    
    return X_output    

tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float",[3,4,4,6])
    X = np.random.randn(3,4,4,6)
    
    A = convolutional_block(A_prev,f=2,filters=[2,4,6],stage=1,block="a")
    test.run(tf.global_variables_initializer())
    
    out = test.run([A],feed_dict={A_prev:X,K.learning_phase():0})
    print("out = " + str(out[0][1][1][0]))
    
    test.close()

def ResNet50(input_shape, classes):
    """
    功能：
    实现resnet50残差网络
    参数:
    input_shape:输入数据的维度(n_H,n_W,n_C)
    classes:分类种类
    返回：
    net:残差网络模型
    """
    X_input = Input(input_shape)
    
    #填充
    X = ZeroPadding2D((3,3))(X_input)
    
    #stage 1
    X = Conv2D(filters = 64, 
               kernel_size=(7,7),
               strides=(2,2),
               name="conv1",
               kernel_initializer=glorot_uniform(seed=0)
              )(X)
    X = BatchNormalization(axis=3, name="bn_conv1")(X)
    X = Activation("relu")(X)
    X = MaxPooling2D(pool_size=(3,3), strides=(2,2))(X)
    
    #stage 2
    X = convolutional_block(X, f=3, filters=[64,64,256], stage=2, block="a", s=1)
    X = identity_block(X, f=3, filters=[64,64,256], stage=2, block="b")
    X = identity_block(X, f=3, filters=[64,64,256], stage=2, block="c")
    
    #stage3
    X = convolutional_block(X, f=3, filters=[128,128,512], stage=3, block="a", s=2)
    X = identity_block(X, f=3, filters=[128,128,512], stage=3, block="b")
    X = identity_block(X, f=3, filters=[128,128,512], stage=3, block="c")
    X = identity_block(X, f=3, filters=[128,128,512], stage=3, block="d")
    
    #stage4
    X = convolutional_block(X, f=3, filters=[256,256,1024], stage=4, block="a", s=2)
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="b")
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="c")
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="d")
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="e")
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block="f")
    
    #stage5
    X = convolutional_block(X, f=3, filters=[512,512,2048], stage=5, block="a", s=2)
    X = identity_block(X, f=3, filters=[512,512,2048], stage=5, block="b")
    X = identity_block(X, f=3, filters=[512,512,2048], stage=5, block="c")
    
    #均匀池化
    X = AveragePooling2D(pool_size=(2,2), padding="same", name="avg_pool")(X) 
    
    X = Flatten()(X)
    X = Dense(classes, 
              activation="softmax", 
              name="fc"+str(classes),
              kernel_initializer=glorot_uniform(seed=0)
             )(X)
    
    model = Model(inputs = X_input, outputs = X, name = "ResNet")
    
    return model

model = ResNet50(input_shape=(64,64,3), classes=6)
model.compile("adam", "binary_crossentropy", metrics=['accuracy'])


out = [0.        0.        0.        0.8929657 0.        0.1988273]
